In [ ]:
import pandas as pd
import numpy as np
import re
from nltk.corpus import stopwords
import re
import nltk
nltk.download('stopwords')

df = pd.read_csv("/content/drive/MyDrive/Sentiments_comp/train_2kmZucJ.csv")

sw = stopwords.words('english')
def clean_text(text):
    text = text.lower()
    text = re.sub(r"[^a-zA-Z?.!,¿]+", " ", text) # replacing everything with space except (a-z, A-Z, ".", "?", "!", ",")
    text = re.sub(r"http\S+", "",text) #Removing URLs
    #text = re.sub(r"http", "",text)
    html=re.compile(r'<.*?>')
    text = html.sub(r'',text) #Removing html tags
    punctuations = '@#!?+&*[]-%.:/();$=><|{}^' + "'`" + '_'

    for p in punctuations:
        text = text.replace(p,'') #Removing punctuations

    text = [word.lower() for word in text.split() if word.lower() not in sw]
    text = " ".join(text) #removing stopwords
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    text = emoji_pattern.sub(r'', text) #Removing emojis
    return text

df['tweet'] = df['tweet'].apply(lambda x: clean_text(x))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
from datasets import Dataset, DatasetDict
from sklearn.model_selection import train_test_split
import pandas as pd

train_df, valid_df = train_test_split(df, test_size=0.2, stratify=df['label'], random_state=42)

# Convert pandas DataFrames to Hugging Face Dataset objects
train_dataset = Dataset.from_pandas(train_df)
valid_dataset = Dataset.from_pandas(valid_df)

# Wrap them in a DatasetDict
dataset = DatasetDict({
    "train": train_dataset,
    "test": valid_dataset
})

dataset["train"] = dataset["train"].remove_columns(["id", "__index_level_0__"])
dataset["test"] = dataset["test"].remove_columns(["id", "__index_level_0__"])

# Inspect the first sample
print(dataset["train"][0])

{'label': 0, 'tweet': 'chocolate cake courtesy miss fara manager samsung office bday http instagramcom p lrshejwn c'}


In [ ]:
from transformers import AutoTokenizer
model_name = "bert-base-uncased"
# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Tokenize the 'tweet' column
def tokenize_function(examples):
    return tokenizer(examples["tweet"], padding="max_length", truncation=True, max_length=256)

# Apply tokenizer
tokenized_datasets = dataset.map(tokenize_function, batched=True)

# Inspect one tokenized example
print(tokenized_datasets["train"][0])

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Map:   0%|          | 0/6336 [00:00<?, ? examples/s]

Map:   0%|          | 0/1584 [00:00<?, ? examples/s]

{'label': 0, 'tweet': 'chocolate cake courtesy miss fara manager samsung office bday http instagramcom p lrshejwn c', 'input_ids': [101, 7967, 9850, 14571, 3335, 2521, 2050, 3208, 19102, 2436, 1038, 10259, 8299, 16021, 23091, 9006, 1052, 1048, 2869, 5369, 3501, 7962, 1039, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'token_type_ids': [0, 0,

In [ ]:
from transformers import AutoModelForSequenceClassification

# Load model
model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=2
)


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from transformers import Trainer, TrainingArguments
import evaluate

accuracy = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return accuracy.compute(predictions=predictions, references=labels)


training_args = TrainingArguments(
    output_dir = f"/content/drive/MyDrive/Sentiments_comp/Model_train",
    do_train=True,
    do_eval=True,
    eval_strategy="steps",
    save_strategy="steps", #no for no saving
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=24,
    learning_rate=5e-5,
    logging_dir="./logs",
    logging_steps=50,
    save_steps=200,
    eval_steps=200,
    save_total_limit=1,
    metric_for_best_model="accuracy",
    greater_is_better=True,
    load_best_model_at_end=True,
    report_to="none",
    fp16=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)


trainer.train()

/tmp/ipython-input-8-1084653421.py:34: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Step,Training Loss,Validation Loss,Accuracy
200,0.151000,0.317689,0.896465
400,0.239400,0.262318,0.907828
600,0.165300,0.343882,0.900884
800,0.147700,0.322396,0.903409
1000,0.087600,0.373917,0.898359


TrainOutput(global_step=1188, training_loss=0.16633670338075168, metrics={'train_runtime': 462.3843, 'train_samples_per_second': 41.109, 'train_steps_per_second': 2.569, 'total_flos': 2500607470141440.0, 'train_loss': 0.16633670338075168, 'epoch': 3.0})

In [ ]:
import torch

test = pd.read_csv("/content/drive/MyDrive/Sentiments_comp/test_oJQbWVk.csv")
test['tweet'] = test['tweet'].apply(lambda x: clean_text(x))

# Tokenize the test set
tokenized_test_dataset = test.map(tokenize_function, batched=True)

# Remove only valid extra columns (not __index_level_0__)
tokenized_test_dataset = tokenized_test_dataset.remove_columns(["id", "tweet"])

# Run prediction
predictions_output = trainer.predict(tokenized_test_dataset)

# Get predicted labels
predicted_labels = np.argmax(predictions_output.predictions, axis=-1)

# Add to the original test DataFrame
test['label'] = predicted_labels

# Save to CSV
test[['id', 'label']].to_csv("/content/drive/MyDrive/Sentiments_comp/submission.csv", index=False)

Map:   0%|          | 0/1953 [00:00<?, ? examples/s]

In [ ]:
trainer.save_model(f"/content/drive/MyDrive/Sentiments_comp/Best_Model")
tokenizer.save_pretrained(f"/content/drive/MyDrive/Sentiments_comp/Best_Model")

('/content/drive/MyDrive/Sentiments_comp/Best_Model/tokenizer_config.json',
 '/content/drive/MyDrive/Sentiments_comp/Best_Model/special_tokens_map.json',
 '/content/drive/MyDrive/Sentiments_comp/Best_Model/vocab.txt',
 '/content/drive/MyDrive/Sentiments_comp/Best_Model/added_tokens.json',
 '/content/drive/MyDrive/Sentiments_comp/Best_Model/tokenizer.json')